# Worksheet 08

Name: Dayu Li 
UID: U61453443

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.774839596547915, 5.706248853136028, 4.306880233289141, 3.869520145120187, 6.918372251953881, 4.589531183609753, 4.486647927987525, 6.312297193194249, 4.897799554082781, 4.545348102607723]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[7.408172166325048, 8.43016797291426, 8.839187020628659, 10.21442097430089, 7.082736249903924, 9.291040102507086, 6.641308727093904, 5.7464039472401085, 8.475825475525564, 6.710961460748394]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[6.710961460748394, 4.545348102607723, 8.475825475525564, 5.7464039472401085, 4.897799554082781, 6.312297193194249, 6.641308727093904, 9.291040102507086, 4.486647927987525, 7.082736249903924]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

1.number of components (K):2
2.means (μ) of each component:5, 8
3.variances (σ²) of each component:1
4.mixture weights (π) of Each Component:0.5

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [6]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]

print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[8.475825475525564, 9.291040102507086]
[6.710961460748394, 4.545348102607723, 5.7464039472401085, 4.897799554082781, 6.312297193194249, 6.641308727093904, 4.486647927987525, 7.082736249903924]
P(S_1) = 0.2,  P(S_2) = 0.8
mean_1 = 8.883432789016325,  mean_2 = 5.802937895357326
var_1 = 0.1661437220111555,  var_2 = 0.9451657548208288


/Users/dayuli/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The number of components and their rough positions (means) are correctly identified.
The estimated proportions deviate from the true mixture weights, which were both 0.5, indicating an equal distribution between the two components. This discrepancy could be due to the random nature of the data generation and selection process.
The mean estimates are relatively close to the true means of 5 and 8. However, the variance estimates are somewhat close but not exact, especially for the second component where the estimated variance is smaller than the true variance of 1 for both components.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [7]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  6.710961460748394
probability of observing that point if it came from cluster 0 =  1.7907084483139213e-37
probability of observing that point if it came from cluster 1 =  0.26606392369541126
point =  4.545348102607723
probability of observing that point if it came from cluster 0 =  2.1840385949540603e-148
probability of observing that point if it came from cluster 1 =  0.17416981413872168
point =  8.475825475525564
probability of observing that point if it came from cluster 0 =  0.11842456600729132
probability of observing that point if it came from cluster 1 =  0.007741093111931622
point =  5.7464039472401085
probability of observing that point if it came from cluster 0 =  9.241027934574888e-78
probability of observing that point if it came from cluster 1 =  0.42133273495985407
point =  4.897799554082781
probability of observing that point if it came from cluster 0 =  2.615305709840589e-125
probability of observing that point if it came from cluster 1 =  0.26684409889505845
p

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [8]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([(x - mean[0])**2 * p for x, p in zip(data, prob_s0_x)]) / sum(prob_s0_x),
    sum([(x - mean[1])**2 * p for x, p in zip(data, prob_s1_x)]) / sum(prob_s1_x)
 ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.2,  P(S_2) = 0.8
mean_1 = 8.927419982439893,  mean_2 = 5.876881258533317
var_1 = 0.16420884882587328,  var_2 = 1.1170894384763785


Ps and means are almost the same, while variance_2 are little bit different. But the values are almost the same.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [14]:
prob_s0_x_updated = []  # P(S_0 | X_i)
prob_s1_x_updated = []  # P(S_1 | X_i)

for x in data:
    pdf_0 = norm.pdf(x, mean[0], np.sqrt(var[0]))
    pdf_1 = norm.pdf(x, mean[1], np.sqrt(var[1]))
    
    total_prob = (pdf_0 * prob_c[0]) + (pdf_1 * prob_c[1])
    
    prob_s0_x_updated.append((pdf_0 * prob_c[0]) / total_prob)
    prob_s1_x_updated.append((pdf_1 * prob_c[1]) / total_prob)

for i, (x, prob_0, prob_1) in enumerate(zip(data, prob_s0_x_updated, prob_s1_x_updated)):
    print(f"Data Point {i+1}: {x:.4f}")
    print(f"  Probability of Cluster 0 (S_0): {prob_0:.6f}")
    print(f"  Probability of Cluster 1 (S_1): {prob_1:.6f}")
    

Data Point 1: 6.7110
  Probability of Cluster 0 (S_0): 0.000000
  Probability of Cluster 1 (S_1): 1.000000
Data Point 2: 4.5453
  Probability of Cluster 0 (S_0): 0.000000
  Probability of Cluster 1 (S_1): 1.000000
Data Point 3: 8.4758
  Probability of Cluster 0 (S_0): 0.861657
  Probability of Cluster 1 (S_1): 0.138343
Data Point 4: 5.7464
  Probability of Cluster 0 (S_0): 0.000000
  Probability of Cluster 1 (S_1): 1.000000
Data Point 5: 4.8978
  Probability of Cluster 0 (S_0): 0.000000
  Probability of Cluster 1 (S_1): 1.000000
Data Point 6: 6.3123
  Probability of Cluster 0 (S_0): 0.000000
  Probability of Cluster 1 (S_1): 1.000000
Data Point 7: 6.6413
  Probability of Cluster 0 (S_0): 0.000000
  Probability of Cluster 1 (S_1): 1.000000
Data Point 8: 9.2910
  Probability of Cluster 0 (S_0): 0.985819
  Probability of Cluster 1 (S_1): 0.014181
Data Point 9: 4.4866
  Probability of Cluster 0 (S_0): 0.000000
  Probability of Cluster 1 (S_1): 1.000000
Data Point 10: 7.0827
  Probability o

On point 10 the Probability becomes less extream than before.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [15]:
hard_assignments = [0 if p_s0 > p_s1 else 1 for p_s0, p_s1 in zip(prob_s0_x_updated, prob_s1_x_updated)]

hard_assignments

[1, 1, 0, 1, 1, 1, 1, 0, 1, 1]